In [1]:
import pandas as pd
import numpy as np
import os
import subprocess
import torch
import json
import sys

In [2]:
# train = pd.read_csv("../../cache/track2/tmp/train.csv")
# val = pd.read_csv('../../cache/track2/tmp/val.csv')
# test = pd.read_csv("../../cache/track2/tmp/test.csv")

## dict to df

In [3]:
with open("../../cache/track2/tmp/face_data.json", "r") as f:
    j = json.load(f)

In [4]:
t = pd.DataFrame.from_dict(j, orient='index')

In [5]:
t = t.reset_index()

In [6]:
t.head()

,index,0,1,2,3,4,5,6,7
0,48318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1192644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1329871,100.0,0.0,1.0,65.0,57.0,30.0,1.0,0.0
3,323042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,864967,100.0,0.0,1.0,47.0,57.0,1.0,1.0,0.0


In [9]:
columns = ["item_id", "male_perc","female_perc","faces","maleBeauty","femaleBeauty","faceSquare", "male_num","femalie_num"]


In [13]:
columns = ["item_id", "male_perc","female_perc","faces","maleBeauty","femaleBeauty","faceSquare", "male_num", "femalie_num"]
print(len(columns))

9


In [10]:
t.columns = columns

In [11]:
t.head()

,item_id,male_perc,female_perc,faces,maleBeauty,femaleBeauty,faceSquare,male_num,femalie_num
0,48318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1192644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1329871,100.0,0.0,1.0,65.0,57.0,30.0,1.0,0.0
3,323042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,864967,100.0,0.0,1.0,47.0,57.0,1.0,1.0,0.0


In [6]:
columns = ["item_id", "title_length"]

In [7]:
columns.extend(["word_"+str(i) for i in range(10)])

In [8]:
t.columns = columns

In [13]:
t = t.astype(int)

In [14]:
t.dtypes

item_id         int64
title_length    int64
word_0          int64
word_1          int64
word_2          int64
word_3          int64
word_4          int64
word_5          int64
word_6          int64
word_7          int64
word_8          int64
word_9          int64
dtype: object

In [9]:
t.head()

,item_id,title_length,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9
0,4036886,6,2,6,0,0,0,0,0,0,0,0
1,2893187,3,8,0,0,0,0,0,0,0,0,0
2,338441,9,16,17,18,10,11,12,14,15,0,0
3,1240180,6,19,20,21,22,24,0,0,0,0,0
4,3284362,9,32,25,26,27,28,29,30,31,0,0


In [10]:
big_data = pd.read_csv("../../cache/track2/tmp/all_data_lgb.csv1")

In [11]:
big_data.shape

(2803192, 236)

In [15]:
tt = big_data.merge(t, on="item_id", how="left")

In [16]:
tt.shape

(2803192, 247)

In [17]:
np.sum(tt[columns].isnull())

item_id              0
title_length    473741
word_0          473741
word_1          473741
word_2          473741
word_3          473741
word_4          473741
word_5          473741
word_6          473741
word_7          473741
word_8          473741
word_9          473741
dtype: int64

In [19]:
tt.fillna(0, inplace=True)

In [20]:
np.sum(tt[columns].isnull())

item_id         0
title_length    0
word_0          0
word_1          0
word_2          0
word_3          0
word_4          0
word_5          0
word_6          0
word_7          0
word_8          0
word_9          0
dtype: int64

In [18]:
t.head()

,item_id,title_length,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9
0,4036886,6,2,6,0,0,0,0,0,0,0,0
1,2893187,3,8,0,0,0,0,0,0,0,0,0
2,338441,9,16,17,18,10,11,12,14,15,0,0
3,1240180,6,19,20,21,22,24,0,0,0,0,0
4,3284362,9,32,25,26,27,28,29,30,31,0,0


In [27]:
tt.head()

,uid,user_city,item_id,author_id,item_city,channel,music_id,device,duration_time,finish,...,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9
0,57384,53,43192,142828,1,0,4514,34178,39,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3230,6,46822,231026,2,0,5331,24878,16,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1249,329,1209078,456220,3,0,39980,14274,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11928,9,1209079,456221,4,0,0,16649,9,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51266,90,1209080,126416,5,0,0,27090,19,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
pp = set(tt.item_id)

In [32]:
len(pp)

862302

In [29]:
ppp = set(t.item_id)

In [33]:
len(ppp)

3114071

In [30]:
len(pp & ppp)

0

In [31]:
tt.item_id.sort_values()

1420645          2
312252           2
1733086         13
2229280         13
979574          13
2700422         13
144741          13
67019           13
2700850         13
1729206         13
988688          13
991891          13
2224648         13
425473          13
1000682         13
1004093         13
1006528         13
1008696         13
236632          13
2594242         13
422249          13
1015252         13
1015751         13
1965170         13
1018225         13
1965201         13
2593887         13
2216968         13
2462668         13
2215703         13
            ...   
2802999    2729279
2803013    2729280
2803018    2729281
2803021    2729282
2803029    2729283
2803030    2729284
2803035    2729285
2803041    2729286
2803043    2729287
2803065    2729288
2803067    2729289
2803069    2729290
2803083    2729291
2803089    2729292
2803096    2729293
2803100    2729294
2803104    2729295
2803107    2729296
2803108    2729297
2803120    2729298
2803126    2729299
2803128    2

In [34]:
t.item_id.sort_values()

2373265          0
2533462          1
768489          10
1776072        100
1036056       1000
544840       10000
713975      100000
737107     1000000
2056821    1000001
141282     1000003
545455     1000004
2103728    1000005
2139580    1000007
2787104    1000008
32421      1000009
972404      100001
2519501    1000010
111090     1000011
2494950    1000012
755637     1000013
3034039    1000014
1822900    1000015
1423818    1000017
1454999    1000018
2701107     100002
328456     1000021
368269     1000022
964617     1000023
2020841    1000025
1085104    1000026
            ...   
2213935     999966
244160      999967
1148911     999968
503455      999969
148529       99997
112821      999970
320616      999971
2408828     999972
790283      999973
2388683     999974
3100803     999975
190768      999976
2540643     999977
3070416     999978
159938       99998
1669843     999980
570026      999981
145206      999982
2986004     999983
284660      999984
566661      999985
1105730     

In [35]:
max(pp)

2729308

In [37]:
min(pp)

2

In [36]:
max(ppp)

'999997'

In [38]:
min(ppp)

'0'